In [ ]:
import os
import cv2
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras import layers, Sequential, datasets, Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import StratifiedKFold
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
import keras.src.saving
import warnings
warnings.filterwarnings('ignore')

In [1]:
# Task 1: Import images with labels and print in a tabular format
data_folder = "/home/dhawi/Documents/dataset"
dataset = data_folder + "/AI_project"
model_folder = "/home/dhawi/Documents/model"
history_folder = "/home/dhawi/Documents/History"

In [2]:
from keras import ops
class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.seed_generator = keras.random.SeedGenerator(1337)

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = ops.shape(z_mean)[0]
        dim = ops.shape(z_mean)[1]
        epsilon = keras.random.normal(shape=(batch, dim), seed=self.seed_generator)
        return z_mean + ops.exp(0.5 * z_log_var) * epsilon

2024-10-11 00:04:59.374495: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-11 00:04:59.382221: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-11 00:04:59.390092: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-11 00:04:59.392427: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-11 00:04:59.399308: I tensorflow/core/platform/cpu_feature_guar

NameError: name 'layers' is not defined

In [ ]:
def generate_images(generator, encoder, decoder, num_images):
    noise = tf.random.normal((num_images, 1024, 1))
    noise = decoder.predict(noise)
    noise = encoder.predict(noise)[2]
    noise = tf.convert_to_tensor(noise)
    generated_images = generator.predict(noise)
    # generated_images = (generated_images * 127.5 + 127.5).astype(np.uint8)  # Rescale to [0, 255]
    return generated_images

In [ ]:
caries_gan_generator = keras.src.saving.load_model(model_folder + "/fyp_caries_EDWGAN1024_encoder.h5", compile=False)
gingivitis_gan_generator = keras.src.saving.load_model(model_folder + "/fyp_gingivitis_EDWGAN1024_encoder.h5", compile=False)
wsl_gan_generator = keras.src.saving.load_model(model_folder + "/fyp_wsls_EDWGAN1024_encoder.h5", compile=False)

In [ ]:
caries_encoder = keras.src.saving.load_model(model_folder + "/fyp_caries_vae1024_encoder.h5", custom_objects={'Sampling': Sampling}, compile=False)
caries_decoder = keras.src.saving.load_model(model_folder + "/fyp_caries_vae1024_decoder.h5", custom_objects={'Sampling': Sampling}, compile=False)
gingivitis_encoder = keras.src.saving.load_model(model_folder + "/fyp_gingivitis_vae1024_encoder.h5", custom_objects={'Sampling': Sampling}, compile=False)
gingivitis_decoder = keras.src.saving.load_model(model_folder + "/fyp_gingivitis_vae1024_decoder.h5", custom_objects={'Sampling': Sampling}, compile=False)
wsl_encoder = keras.src.saving.load_model(model_folder + "/fyp_wsls_vae1024_encoder.h5", custom_objects={'Sampling': Sampling}, compile=False)
wsl_decoder = keras.src.saving.load_model(model_folder + "/fyp_wsls_vae1024_decoder.h5", custom_objects={'Sampling': Sampling}, compile=False)

In [ ]:
num_images_to_generate = 10

generated_caries_images = generate_images(caries_gan_generator, caries_encoder, caries_decoder, num_images_to_generate)
generated_gingivitis_images = generate_images(gingivitis_gan_generator, gingivitis_encoder, gingivitis_decoder, num_images_to_generate)
generated_wsl_images = generate_images(wsl_gan_generator, wsl_encoder, wsl_decoder, num_images_to_generate)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 696ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 129ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 634ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 129ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 680ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step


In [ ]:
# Load the trained ResNet model
trained_resnet_model = tf.keras.models.load_model('/home/manjoe/FYP/models/keras_best_model.keras')

def classify_images(model, images):
    predictions = model.predict(images)
    confidences = np.max(predictions, axis=1)
    predicted_labels = np.argmax(predictions, axis=1)
    return predicted_labels, confidences